<a href="https://colab.research.google.com/github/ankitsrivastava637/text-similarity---contradictory-statements-detection/blob/main/text_similarity_and_contradictory_statements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sentence_transformers import SentenceTransformer, util
import time



model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

start = time.time()
# Single list of sentences - Possible tens of thousands of sentences
sentences = ['The cat sits outside',
             'A man is playing guitar',
             'I love pasta',
             'The new movie is awesome',
             'The cat plays in the garden',
             'A woman watches TV',
             'The new movie is so great',
             'Do you like pizza?']

paraphrases = util.paraphrase_mining(model, sentences)

for paraphrase in paraphrases[0:10]:
       score, i, j = paraphrase
       if sentences[i] == 'The cat sits outside' and score>0.70:
           print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))
           print
           break
       else:
           print('Request handover')  
           break  
end = time.time()    
print(end-start)

Request handover
0.11644530296325684


In [ ]:
#Explacy

import sys
from collections import defaultdict
import spacy
from nltk.corpus import wordnet
from spacy import displacy
from collections import Counter
import en_core_web_sm
from nltk.tokenize import word_tokenize
import nltk 
nltk.download('wordnet')
from nltk.corpus import wordnet

from pprint import pprint

_do_print_debug_info = False





#Taking the sentences as input

nlp = en_core_web_sm.load()
sent1="""We are a finance company. 
          I like Python because I can build applications,
          We are trying to give best service.
          The cat sits on the ground.
          We do not accept credit cards. 
         The cat walks on the sidewalk"""
sent2="We only accept credit cards."

#print("\n")

start = time.time()
doc1 = nlp(sent1)
doc2 = nlp(sent2)


#Antonyms finder function
synonyms = []
antonyms = []
#word="went"
def antysyn(word):
    for syn in wordnet.synsets(word):
        #print(syn)
        for l in syn.lemmas():
            #print(l)
            synonyms.append(l.name())
            if l.antonyms():
                antonyms.append(l.antonyms()[0].name())
    #print("Synonym:",set(synonyms))
    #print("Antonym:",set(antonyms))   


#Initializing required variables and lists.
wrdlist=list()
antony=list()
contr_tracker=0
antonym_tracker=0

negdoc1=0
negdoc2=0
verb1=""
verb2=""
num_contr_tracker=0


#Processing sentence 1
for token in doc1:
    if(token.dep_=="neg"):
        negdoc1=1
        verb1+="NOT "
    #Storing the antonyms of root words
    if(token.pos_=="VERB" and token.dep_=="ROOT"):
        #print(token.text)
        verb1+=token.lemma_
        antysyn(token.lemma_)
        for anton in antonyms:
            antony.append(anton)


#Processing Sentence 2
for token in doc2:
    if(token.dep_=="neg"):
        negdoc2=1
        verb2+="NOT "
    if(token.pos_=="VERB" and token.dep_=="ROOT"):
        verb2+=token.lemma_
        if(token.lemma_ in antony):
            antonym_tracker=1



#Function for checking negation
def checknegationcontradiction(antonym_tracker,negdoc1,negdoc2):
    temp_var=negdoc1+negdoc2+antonym_tracker
    if(temp_var%2!=0 and temp_var<3):
        return 1
    else:
        return 0

#Checking contradiction due to negation
contr_tracker=checknegationcontradiction(antonym_tracker,negdoc1,negdoc2)


def check_words(doc):
    merged_word=""
    #tokens = word_tokenize(doc)
    WordNum  = len(doc)
    #print("WordNum is:"+str(WordNum))
    for i in range(WordNum):
        #print(i,doc[i],doc[i].ent_type_)
        #print(str.isdigit(doc[i].text))
        if(doc[i].ent_type_=="CARDINAL" or doc[i].pos_=="NUM"):
             merged_word = doc[i].text
        if((doc[i].ent_type_=="CARDINAL" or doc[i].pos_=="NUM") and str.isdigit(doc[i].text)):
            if(not(str.isdigit(doc[i-1].text))):
                if(not(str.isdigit(doc[i-2].text))):
                    merged_word= doc[i-2].text+' '+doc[i-1].text+' '+doc[i].text
    return merged_word


def check_values(t1,t2):
    for phrase in checklist_more:
        if(t1.find(phrase)!=-1 and t2.find(phrase)==-1):
            num1 = t1.replace(phrase,'')
            num2 = [int(s) for s in str.split(t2) if s.isdigit()]
            num2=num2[0]
            if int(num1)>num2:
                print("case1")
                return('Contradiction')
            else:
                return("No Contradiction")
        else:
            return("No Contradiction")
    for phrase in checklist_more:
        if(t2.find(phrase)!=-1 and t1.find(phrase)==-1):
            num2 = t2.replace(phrase,'')
            num1 = [int(s) for s in str.split(t2) if s.isdigit()]
            num1=num1[0]
            if int(num2)>num1:
                print("case2")
                return('Contradiction') 
            else:
                return("No Contradiction")
        else:
            return("No Contradiction")
    for phrase in checklist_less:
        if(t1.find(phrase)!=-1 and t2.find(phrase)==-1):
            num1 = t1.replace(phrase,'')
            num2 = [int(s) for s in str.split(t2) if s.isdigit()]
            num2=num2[0]
            if int(num1)<num2:
                print("case3")
                return('Contradiction')
            else:
                return("No Contradiction")
        else:
                return("No Contradiction")
    for phrase in checklist_less:
        if(t2.find(phrase)!=-1 and t1.find(phrase)==-1):
            num2 = t2.replace(phrase,'')
            num1 = [int(s) for s in str.split(t2) if s.isdigit()]
            num1=num1[0]
            if int(num1)>num2:
                print("case4")
                return('Contradiction')
            else: 
                return("No Contradiction")
        else:
                return("No Contradiction")
    else:
        if(t1!=t2):
            return('Contradiction') 

#Finding numerical mismatch
checklist_more=['more than ', 'greater than ', 'above', 'all']
checklist_less=['less than ', 'lesser than ', 'below', 'only']


x1 = check_words(doc1)
#print(x1)
y1 = check_words(doc2)
#print("Second Sentence:",y1)
number_contr_tracker=check_values(x1,y1)
if(number_contr_tracker=='Contradiction'):
    num_contr_tracker=1
else:
    num_contr_tracker=0


if contr_tracker==1:
    print("\n","->",verb1.upper(),"and",verb2.upper(),"can't happen simultaneously.")
    print("->Antonymity/Negation contradiction FOUND.")
else:
    print("\n->Antonymity/Negation contradiction NOT found.")

if num_contr_tracker==1:
    print("->Numeric Mismatch Contradiction FOUND.")
else:
    print("->Numeric Mismatch Contradiction NOT Found.")    

end = time.time()
print("total time taken:", end-start)    

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!

 -> LIKETRYSITNOT ACCEPTWALK and ACCEPT can't happen simultaneously.
->Antonymity/Negation contradiction FOUND.
->Numeric Mismatch Contradiction NOT Found.
total time taken: 0.14984536170959473


In [ ]:
import time
#Explacy

import sys
from collections import defaultdict
import spacy
from nltk.corpus import wordnet
from spacy import displacy
from collections import Counter
import en_core_web_sm
from nltk.tokenize import word_tokenize
import nltk 
nltk.download('wordnet')
from nltk.corpus import wordnet

from pprint import pprint

_do_print_debug_info = False





#Taking the sentences as input

nlp = en_core_web_sm.load()




class ContradictionFilter:
    def __init__(self, context, reply):
        self.context = context
        self.reply = reply


    def contradiction_detection(self):

        sent1=self.context
        sent2=self.reply
        
    #print("\n")

        start = time.time()
        doc1 = nlp(sent1)
        doc2 = nlp(sent2)
        
        
        #Antonyms finder function
        synonyms = []
        antonyms = []
        #word="went"
        def antysyn(word):
            for syn in wordnet.synsets(word):
                #print(syn)
                for l in syn.lemmas():
                    #print(l)
                    synonyms.append(l.name())
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())
            #print("Synonym:",set(synonyms))
            #print("Antonym:",set(antonyms))   
        
        
        #Initializing required variables and lists.
        wrdlist=list()
        antony=list()
        contr_tracker=0
        antonym_tracker=0
        
        negdoc1=0
        negdoc2=0
        verb1=""
        verb2=""
        num_contr_tracker=0
        
        
        #Processing sentence 1
        for token in doc1:
            if(token.dep_=="neg"):
                negdoc1=1
                verb1+="NOT "
            #Storing the antonyms of root words
            if(token.pos_=="VERB" and token.dep_=="ROOT"):
                #print(token.text)
                verb1+=token.lemma_
                antysyn(token.lemma_)
                for anton in antonyms:
                    antony.append(anton)
        
        
        #Processing Sentence 2
        for token in doc2:
            if(token.dep_=="neg"):
                negdoc2=1
                verb2+="NOT "
            if(token.pos_=="VERB" and token.dep_=="ROOT"):
                verb2+=token.lemma_
                if(token.lemma_ in antony):
                    antonym_tracker=1
        
        
        
        #Function for checking negation
        def checknegationcontradiction(antonym_tracker,negdoc1,negdoc2):
            temp_var=negdoc1+negdoc2+antonym_tracker
            if(temp_var%2!=0 and temp_var<3):
                return 1
            else:
                return 0
        
        #Checking contradiction due to negation
        contr_tracker=checknegationcontradiction(antonym_tracker,negdoc1,negdoc2)
        
        
        def check_words(doc):
            merged_word=""
            #tokens = word_tokenize(doc)
            WordNum  = len(doc)
            #print("WordNum is:"+str(WordNum))
            for i in range(WordNum):
                #print(i,doc[i],doc[i].ent_type_)
                #print(str.isdigit(doc[i].text))
                if(doc[i].ent_type_=="CARDINAL" or doc[i].pos_=="NUM"):
                    merged_word = doc[i].text
                if((doc[i].ent_type_=="CARDINAL" or doc[i].pos_=="NUM") and str.isdigit(doc[i].text)):
                    if(not(str.isdigit(doc[i-1].text))):
                        if(not(str.isdigit(doc[i-2].text))):
                            merged_word= doc[i-2].text+' '+doc[i-1].text+' '+doc[i].text
            return merged_word
        
        
        def check_values(t1,t2):
            for phrase in checklist_more:
                if(t1.find(phrase)!=-1 and t2.find(phrase)==-1):
                    num1 = t1.replace(phrase,'')
                    num2 = [int(s) for s in str.split(t2) if s.isdigit()]
                    num2=num2[0]
                    if int(num1)>num2:
                        print("case1")
                        return('Contradiction')
                    else:
                        return("No Contradiction")
                else:
                    return("No Contradiction")
            for phrase in checklist_more:
                if(t2.find(phrase)!=-1 and t1.find(phrase)==-1):
                    num2 = t2.replace(phrase,'')
                    num1 = [int(s) for s in str.split(t2) if s.isdigit()]
                    num1=num1[0]
                    if int(num2)>num1:
                        print("case2")
                        return('Contradiction') 
                    else:
                        return("No Contradiction")
                else:
                    return("No Contradiction")
            for phrase in checklist_less:
                if(t1.find(phrase)!=-1 and t2.find(phrase)==-1):
                    num1 = t1.replace(phrase,'')
                    num2 = [int(s) for s in str.split(t2) if s.isdigit()]
                    num2=num2[0]
                    if int(num1)<num2:
                        print("case3")
                        return('Contradiction')
                    else:
                        return("No Contradiction")
                else:
                        return("No Contradiction")
            for phrase in checklist_less:
                if(t2.find(phrase)!=-1 and t1.find(phrase)==-1):
                    num2 = t2.replace(phrase,'')
                    num1 = [int(s) for s in str.split(t2) if s.isdigit()]
                    num1=num1[0]
                    if int(num1)>num2:
                        print("case4")
                        return('Contradiction')
                    else: 
                        return("No Contradiction")
                else:
                        return("No Contradiction")
            else:
                if(t1!=t2):
                    return('Contradiction') 
        
        #Finding numerical mismatch
        checklist_more=['more than ', 'greater than ', 'above', 'all', 'all']
        checklist_less=['less than ', 'lesser than ', 'below', 'only', 'few']
        
        
        x1 = check_words(doc1)
        #print(x1)
        y1 = check_words(doc2)
        #print("Second Sentence:",y1)
        number_contr_tracker=check_values(x1,y1)
        if(number_contr_tracker=='Contradiction'):
            num_contr_tracker=1
        else:
            num_contr_tracker=0
        
        if contr_tracker==0 and num_contr_tracker==0:
            return self.reply
        
        if contr_tracker==1 or num_contr_tracker==1:
            #print("\n","->",verb1.upper(),"and",verb2.upper(),"can't happen simultaneously.")
            #print("->Antonymity/Negation contradiction FOUND.")
            return 'Reuest Handover'
        else:
            print("\n->Antonymity/Negation contradiction NOT found.")
            return self.reply
        
        if num_contr_tracker==1:
            print("->Numeric Mismatch Contradiction FOUND.")
            #return 'Request Handover'
        else:
            print("->Numeric Mismatch Contradiction NOT Found.")    
            #return self.reply
        end = time.time()

        print("total time taken:", end-start) 
           

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
context = """We are a finance company. 
             I like Python because I can build applications,
             We are trying to give best service.
             The cat sits on the ground.
             We do not accept credit cards. 
             The cat walks on the sidewalk"""
cont = "We accept all credit cards."
reply = "We don't accept cash."
text_infer_obj = ContradictionFilter(context, reply)
text_infer = text_infer_obj.contradiction_detection()   #gives entailment, contradiction, neutral scores between context and response
print(text_infer)

We don't accept cash.


In [ ]:
!pip install transformers
!pip install sentence_transformers

     |████████████████████████████████| 2.3MB 29.4MB/s 
     |████████████████████████████████| 3.3MB 32.9MB/s 
     |████████████████████████████████| 901kB 40.1MB/s 
     |████████████████████████████████| 81kB 10.7MB/s 
     |████████████████████████████████| 1.2MB 40.3MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=9185c6786d9b4ec294fdf96603efe2bfc96310568102aec30eec2e69f6a72143
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
import sys
from collections import defaultdict
import spacy
from nltk.corpus import wordnet
from spacy import displacy
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk 
nltk.download('wordnet')
import time
nltk.download('punkt')


_do_print_debug_info = False

!python -m spacy download en_core_web_md

nlp =  spacy.load("en_core_web_md")

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

start = time.time()

class ResponseFilter:
    def __init__(self, context, reply):
        self.context = context
        self.reply = reply

    def textual_similarity(self):
         # Single list of sentences - Possible tens of thousands of sentences
        context_reply = (self.context + ' ' + self.reply)
        sent_list = nltk.tokenize.sent_tokenize(context_reply)
        sentences = sent_list
        #print(sentences)
        index_of_reply = len(self.reply)

        paraphrases = util.paraphrase_mining(model, sentences)
        #print(paraphrases)
        lst = []

        for paraphrase in paraphrases[0:10]:
           score, i, j = paraphrase
           #print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))
           #print('parAPHRASE:',paraphrase)
           #print(i)
          #print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))
           lst.append([sentences[i], sentences[j], score])
        #print(lst)
          
        for ls in lst:
          #print(ls[0])
          #print(sentences[len(sentences)-1])
          if ls[0] == sentences[len(sentences)-1] or ls[1] == sentences[len(sentences)-1]:
            #print(sentences[len(sentences)-1])
            #print(ls[0])
            if  ls[2] > 0.65:
                text_sim_obj = ContradictionFilter(self.context, self.reply)
                text_sim = text_sim_obj.contradiction_detection()   #gives entailment, contradiction, neutral scores between context and response
                return text_sim
            else:
              return 'Request Handover'
          else:
            continue        
    

end = time.time()    
print(end-start)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
0.0002956390380859375


In [ ]:
start = time.time()
context = """We are a finance company. 
             I like Python because I can build applications.
             We are trying to give best service.
             We do accept credit cards. 
             The cat walks on the sidewalk.
             Credit cards are Capital One’s main physical product, and teams all over the company work to support this product whether it’s in the form of product development, software engineering, design, etc. For example, Alana is a data engineer working on rewards applications for external partners while Sahana works to support data scientists developing machine learning models used to derive insights from card customer calls. Like many others at Capital One, we know what needs to be done to support credit card initiatives, but we were curious about how physical credit cards actually work!
             """
cont = "We accept credit card."
reply = "We accept credit cards." 
print(len(context.split(' ')))     
text_infer_obj = ResponseFilter(context, reply)   
text_infer = text_infer_obj.textual_similarity()   #gives entailment, contradiction, neutral scores between context and response
print(text_infer)
end = time.time()
print('time taken:', end-start)

199
We accept credit cards.
time taken: 0.05808424949645996


In [ ]:
from sentence_transformers import SentenceTransformer, util
import sys
from collections import defaultdict
import spacy
from nltk.corpus import wordnet
from spacy import displacy
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk 
nltk.download('wordnet')
import time
nltk.download('punkt')


_do_print_debug_info = False

!python -m spacy download en_core_web_lg

nlp =  spacy.load("en_core_web_lg")

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

start = time.time()

class ResponseFilter:
    def __init__(self, context, reply):
        self.context = context
        self.reply = reply

    def textual_similarity(self):
         # Single list of sentences - Possible tens of thousands of sentences
        context_reply = (self.context + ' ' + self.reply)
        sent_list = nltk.tokenize.sent_tokenize(context_reply)
        sentences = sent_list
        #print(sentences)
        index_of_reply = len(self.reply)

        paraphrases = util.paraphrase_mining(model, sentences)
        #print(paraphrases)
        lst = []

        for paraphrase in paraphrases[0:10]:
           score, i, j = paraphrase
           print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))
           #print('parAPHRASE:',paraphrase)
           #print(i)
          #print("{} \t\t {} \t\t Score: {:.4f}".format(sentences[i], sentences[j], score))
           lst.append([sentences[i], sentences[j], score])
        #print(lst)
          
        for ls in lst:
          #print(ls[0])
          #print(sentences[len(sentences)-1])
          if ls[0] == sentences[len(sentences)-1] or ls[1] == sentences[len(sentences)-1]:
            #print(sentences[len(sentences)-1])
            #print(ls[0])
            if  ls[2] > 0.65:
                text_sim_obj = ContradictionFilter(self.context, self.reply)
                text_sim = text_sim_obj.contradiction_detection()   #gives entailment, contradiction, neutral scores between context and response
                return text_sim
            else:
              return 'Request Handover'
          else:
            continue        
    

end = time.time()    
print(end-start)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
0.00036597251892089844


In [ ]:
start = time.time()
context = """We are a finance company. 
             I like Python because I can build applications.
             We are trying to give best service.
             We do accept credit cards. 
             The cat walks on the sidewalk.
             Credit cards are Capital One’s main physical product, and teams all over the company work to support this product whether it’s in the form of product development, software engineering, design, etc. For example, Alana is a data engineer working on rewards applications for external partners while Sahana works to support data scientists developing machine learning models used to derive insights from card customer calls. Like many others at Capital One, we know what needs to be done to support credit card initiatives, but we were curious about how physical credit cards actually work!
             We compiled all of our research on EMV chip credit cards and shared it with our fellow Capital One associates during the company’s largest internal Software Engineering Conference (SECON). We were excited to see Software Engineers all over the company learning from our talk, which broke down EMV technology concepts into easily digestible components. Today, our Intro to EMV technology video is still the most-viewed and most-endorsed of all SECON 2020 videos.
             Before we go further into EMV chip technology, let’s first talk about magnetic stripe technology. Magnetic stripes are a predecessor of EMV chips, so it’s helpful to understand how they work and some of the security concerns associated with these cards, which were later resolved by EMV chips.
             In an EMV chip the access and manipulation of the physical chip components is a complex task that requires expensive high-tech equipment. Unlike the static data on a magnetic stripe, the data on an EMV chip is constantly changing which makes it difficult to isolate and extract. Additionally, EMV chips have the benefit of built-in encryption. When the chip card is dipped, it establishes an encrypted connection with the terminal to enforce the transaction rules written on the chip. Both the card and the terminal then collaborate to determine the outcome of the transaction.
             When we started learning about EMV technology, we were fascinated by the complex, yet elegant design of the technology. Admittedly, the concepts were difficult for us to grasp at first, but as we continued our research and consulted internal subject matter experts about this technology, we were able to break down these concepts into understandable components that allowed us to solidify our understanding of Capital One’s main product. We hope that this breakdown of EMV chip technology helped provide insights into the fascinating technology powering 80.1% of worldwide credit card transactions today!
             Also, magnetic stripe cards don’t have encryption. The card is simply a data store that is read by the payment terminal as-is. After the terminal reads the card, the terminal performs all the processing in collaboration with the issuer and/or payment system and the card is no longer involved in the rest of the process."""
cont = "We accept credit card."
reply = "We accept cash." 
print(len(context.split(' ')))     
text_infer_obj = ResponseFilter(context, reply)   
text_infer = text_infer_obj.textual_similarity()   #gives entailment, contradiction, neutral scores between context and response
print(text_infer)
end = time.time()
print('time taken:', end-start)

607
Before we go further into EMV chip technology, let’s first talk about magnetic stripe technology. 		 Magnetic stripes are a predecessor of EMV chips, so it’s helpful to understand how they work and some of the security concerns associated with these cards, which were later resolved by EMV chips. 		 Score: 0.6808
When we started learning about EMV technology, we were fascinated by the complex, yet elegant design of the technology. 		 Admittedly, the concepts were difficult for us to grasp at first, but as we continued our research and consulted internal subject matter experts about this technology, we were able to break down these concepts into understandable components that allowed us to solidify our understanding of Capital One’s main product. 		 Score: 0.6677
Magnetic stripes are a predecessor of EMV chips, so it’s helpful to understand how they work and some of the security concerns associated with these cards, which were later resolved by EMV chips. 		 Unlike the static data on

In [ ]:
import time
from sentence_transformers import SentenceTransformer, util
import sys
from collections import defaultdict
import spacy
from nltk.corpus import wordnet
from spacy import displacy
from collections import Counter
import en_core_web_sm
from nltk.tokenize import word_tokenize
import nltk 
nltk.download('wordnet')
from nltk.corpus import wordnet
import nltk
nltk.download('punkt')

from pprint import pprint

_do_print_debug_info = False

!python -m spacy download en_core_web_md

nlp =  spacy.load("en_core_web_md")

model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')




class ContradictionFilter:
    def __init__(self, context, reply):
        self.context = context
        self.reply = reply


    def contradiction_detection(self):

        sent1=self.context
        sent2=self.reply
        
        doc1 = nlp(sent1)
        doc2 = nlp(sent2)
        
        
        #Antonyms finder function
        synonyms = []
        antonyms = []
        def antysyn(word):
            for syn in wordnet.synsets(word):
                for l in syn.lemmas():
                    synonyms.append(l.name())
                    if l.antonyms():
                        antonyms.append(l.antonyms()[0].name())  
        
        
        #Initializing required variables and lists.
        wrdlist=list()
        antony=list()
        contr_tracker=0
        antonym_tracker=0
        
        negdoc1=0
        negdoc2=0
        verb1=""
        verb2=""
        num_contr_tracker=0
        
        
        #Processing sentence 1
        for token in doc1:
            if(token.dep_=="neg"):
                negdoc1=1
                verb1+="NOT "
            #Storing the antonyms of root words
            if(token.pos_=="VERB" and token.dep_=="ROOT"):
                verb1+=token.lemma_
                antysyn(token.lemma_)
                for anton in antonyms:
                    antony.append(anton)
        
        
        #Processing Sentence 2
        for token in doc2:
            if(token.dep_=="neg"):
                negdoc2=1
                verb2+="NOT "
            if(token.pos_=="VERB" and token.dep_=="ROOT"):
                verb2+=token.lemma_
                if(token.lemma_ in antony):
                    antonym_tracker=1
        
        
        
        #Function for checking negation
        def checknegationcontradiction(antonym_tracker,negdoc1,negdoc2):
            temp_var=negdoc1+negdoc2+antonym_tracker
            if(temp_var%2!=0 and temp_var<3):
                return 1
            else:
                return 0
        
        #Checking contradiction due to negation
        contr_tracker=checknegationcontradiction(antonym_tracker,negdoc1,negdoc2)
        
        
        def check_words(doc):
            merged_word=""
            WordNum  = len(doc)
            for i in range(WordNum):
                if(doc[i].ent_type_=="CARDINAL" or doc[i].pos_=="NUM"):
                    merged_word = doc[i].text
                if((doc[i].ent_type_=="CARDINAL" or doc[i].pos_=="NUM") and str.isdigit(doc[i].text)):
                    if(not(str.isdigit(doc[i-1].text))):
                        if(not(str.isdigit(doc[i-2].text))):
                            merged_word= doc[i-2].text+' '+doc[i-1].text+' '+doc[i].text
            return merged_word
        
        
        def check_values(t1,t2):
            for phrase in checklist_more:
                if(t1.find(phrase)!=-1 and t2.find(phrase)==-1):
                    num1 = t1.replace(phrase,'')
                    num2 = [int(s) for s in str.split(t2) if s.isdigit()]
                    num2=num2[0]
                    if int(num1)>num2:
                        print("case1")
                        return('Contradiction')
                    else:
                        return("No Contradiction")
                else:
                    return("No Contradiction")
            for phrase in checklist_more:
                if(t2.find(phrase)!=-1 and t1.find(phrase)==-1):
                    num2 = t2.replace(phrase,'')
                    num1 = [int(s) for s in str.split(t2) if s.isdigit()]
                    num1=num1[0]
                    if int(num2)>num1:
                        print("case2")
                        return('Contradiction') 
                    else:
                        return("No Contradiction")
                else:
                    return("No Contradiction")
            for phrase in checklist_less:
                if(t1.find(phrase)!=-1 and t2.find(phrase)==-1):
                    num1 = t1.replace(phrase,'')
                    num2 = [int(s) for s in str.split(t2) if s.isdigit()]
                    num2=num2[0]
                    if int(num1)<num2:
                        print("case3")
                        return('Contradiction')
                    else:
                        return("No Contradiction")
                else:
                        return("No Contradiction")
            for phrase in checklist_less:
                if(t2.find(phrase)!=-1 and t1.find(phrase)==-1):
                    num2 = t2.replace(phrase,'')
                    num1 = [int(s) for s in str.split(t2) if s.isdigit()]
                    num1=num1[0]
                    if int(num1)>num2:
                        print("case4")
                        return('Contradiction')
                    else: 
                        return("No Contradiction")
                else:
                        return("No Contradiction")
            else:
                if(t1!=t2):
                    return('Contradiction') 
        
        #Finding numerical mismatch
        checklist_more=['more than ', 'greater than ', 'above', 'all']
        checklist_less=['less than ', 'lesser than ', 'below', 'few']
        
        
        x1 = check_words(doc1)
        y1 = check_words(doc2)
        number_contr_tracker=check_values(x1,y1)
        if(number_contr_tracker=='Contradiction'):
            num_contr_tracker=1
        else:
            num_contr_tracker=0
        
        if contr_tracker==0 and num_contr_tracker==0:
            return self.reply
        
        if contr_tracker==1 or num_contr_tracker==1:
            return 'Request Handover'



class ResponseFilter:
    def __init__(self, context, reply):
        self.context = context
        self.reply = reply

    def textual_similarity(self):
         # Single list of sentences - Possible tens of thousands of sentences
        context_reply = (self.context + ' ' + self.reply)
        sent_list = nltk.tokenize.sent_tokenize(context_reply)
        sentences = sent_list
        first_half_context, second_half_context = self.context[:len(self.context)//2], self.context[len(self.context)//2:]

        paraphrases = util.paraphrase_mining(model, sentences)
        lst = []

        for paraphrase in paraphrases[0:100]:
           score, i, j = paraphrase
           lst.append([sentences[i], sentences[j], score])
          
        for ls in lst:
          print('sim score :   ', ls[2])
          if ls[0] == sentences[len(sentences)-1]:
            if 0.75 < ls[2] < 0.95:
                  text_sim_obj = ContradictionFilter(ls[1], self.reply)
                  text_sim = text_sim_obj.contradiction_detection()   
                  return text_sim       
            elif ls[2]>=0.95:
                  return self.reply 
            else:
                  return 'Request Handover'

          elif ls[1] == sentences[len(sentences)-1]:
            if 0.75 < ls[2] < 0.95:
                  text_sim_obj = ContradictionFilter(ls[1], self.reply)
                  text_sim = text_sim_obj.contradiction_detection() 
                  return text_sim          
            elif ls[2]>=0.95:
                  return self.reply 
            else:
                  return 'Request Handover'         
          else:
            continue  


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
start = time.time()
context = """We are a finance company. 
             I like Python because I can build applications.
             We are trying to give best service.
             We do accept credit cards. 
             The cat walks on the sidewalk.
             A credit card is a payment card issued to users (cardholders) to enable the cardholder to pay a merchant for goods and services based on the cardholder's accrued debt (i.e., promise to the card issuer to pay them for the amounts plus the other agreed charges). The card issuer (usually a bank) creates a revolving account and grants a line of credit to the cardholder, from which the cardholder can borrow money for payment to a merchant or as a cash advance.
             This sometimes led to a case of mistaken identity, either accidentally or intentionally, by acting on behalf of the charge account owner or out of malice to defraud both the charge account owner and the merchant. Beginning in the 1930s, merchants started to move from charge coins to the newer Charga-Plate. Credit card numbers were originally embossed to allow easy transfer of the number to charge slips.
             Credit cards are Capital One’s main physical product, and teams all over the company work to support this product whether it’s in the form of product development, software engineering, design, etc. For example, Alana is a data engineer working on rewards applications for external partners while Sahana works to support data scientists developing machine learning models used to derive insights from card customer calls. Like many others at Capital One, we know what needs to be done to support credit card initiatives, but we were curious about how physical credit cards actually work!
             We compiled all of our research on EMV chip credit cards and shared it with our fellow Capital One associates during the company’s largest internal Software Engineering Conference (SECON). We were excited to see Software Engineers all over the company learning from our talk, which broke down EMV technology concepts into easily digestible components. Today, our Intro to EMV technology video is still the most-viewed and most-endorsed of all SECON 2020 videos.
             Before we go further into EMV chip technology, let’s first talk about magnetic stripe technology. Magnetic stripes are a predecessor of EMV chips, so it’s helpful to understand how they work and some of the security concerns associated with these cards, which were later resolved by EMV chips.
             In an EMV chip the access and manipulation of the physical chip components is a complex task that requires expensive high-tech equipment. Unlike the static data on a magnetic stripe, the data on an EMV chip is constantly changing which makes it difficult to isolate and extract. Additionally, EMV chips have the benefit of built-in encryption. When the chip card is dipped, it establishes an encrypted connection with the terminal to enforce the transaction rules written on the chip. Both the card and the terminal then collaborate to determine the outcome of the transaction.
             When we started learning about EMV technology, we were fascinated by the complex, yet elegant design of the technology. Admittedly, the concepts were difficult for us to grasp at first, but as we continued our research and consulted internal subject matter experts about this technology, we were able to break down these concepts into understandable components that allowed us to solidify our understanding of Capital One’s main product. We hope that this breakdown of EMV chip technology helped provide insights into the fascinating technology powering 80.1% of worldwide credit card transactions today!
             Also, magnetic stripe cards don’t have encryption. The card is simply a data store that is read by the payment terminal as-is. After the terminal reads the card, the terminal performs all the processing in collaboration with the issuer and/or payment system and the card is no longer involved in the rest of the process."""
cont = "We accept credit card."
reply = "We accept credit cards and cash." 
print(len(context.split(' ')))        
text_infer_obj = ResponseFilter(context, reply)
text_infer = text_infer_obj.textual_similarity()   #gives entailment, contradiction, neutral scores between context and response
print(text_infer)
end = time.time()
print('time taken:', end-start)

780
sim score :    0.8328469395637512
We accept credit cards and cash.
time taken: 0.10916733741760254


In [ ]:
# Python3 code to demonstrate working of
# Splitting string into equal halves
# Using string slicing

# initializing string
test_str = "Geeks for Geeks"

# printing original string
print("The original string is : " + test_str)

# Using string slicing
# Splitting string into equal halves
res_first, res_second = test_str[:len(test_str)//2],test_str[len(test_str)//2:]

# printing result
print("The first part of string : " + res_first)
print("The second part of string : " + res_second)


The original string is : Geeks for Geeks
The first part of string : Geeks f
The second part of string : or Geeks


In [ ]:
str = 'The second part of string'
print(str.split())

['The', 'second', 'part', 'of', 'string']
